# StarcNet demo for NGC1566
This notebook executes a demo with **already downloaded** `.fits` and `.tabs` files saved to folders `legus/frc_fits_files` and `legus/tab_files` respectively.

**Note:** To install the starcnet environment kernel to Jupyter run in your terminal:
```
python -m ipykernel install --user --name starcnet --display-name "starcnet"
```

## Table of contents
* [Getting started](#Getting-started)
* [Create dataset](#Create-dataset)
* [Run StarcNet](#Run-StarcNet)
    * [Load dataset](#Load-dataset)
    * [Classify objects](#Classify-objects)
* [Create text file with predictions](#Create-text-file-with-predictions)
* [Create galaxy image with predictions](#Create-galaxy-image-with-predictions)

## Getting started

In [ ]:
import os
import sys
import csv
import time
import pickle
import numpy as np
from numpy.linalg import norm

import torch
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

sys.path.insert(0, './src/utils')
sys.path.insert(0, './model')
from data_utils import load_db
from starcnet import Net

Check that galaxy name and filenames are already in the `.txt` files `targets.txt` and `frc_fits_links.txt`. The complete download link should be added if used with online LEGUS catalogs.

In [ ]:
print(open('targets.txt', 'r').read())
print(open('frc_fits_links.txt', 'r').read())

## Create dataset
Create a 32x32x5 array per object in the catalog (`.tab` file). Each array is object centered and has the 5 bands of the photometric information from the HST.

In [ ]:
print('creating dataset...')
os.system('bash create_dataset.sh')
print('dataset created')

## Run StarcNet

In [ ]:
batch_size = 64 # input batch size for testing (default: 64)
data_dir = 'data/' # dataset directory
dataset = 'raw_32x32' # dataset file reference
checkpoint = 'model/starcnet.pth' # trained model
gpu = '' # CUDA visible device (when using a GPU add GPU id (e.g. '0'))
cuda = False # enables CUDA training (when using a GPU change to True)

### Load dataset

In [ ]:
data_all, _, ids = load_db(os.path.join(data_dir,'test_'+dataset+'.dat'))
mean = np.load(os.path.join(data_dir,'mean.npy'))

data_test = data_all - mean[np.newaxis,:,np.newaxis,np.newaxis] # subtract mean

data = torch.from_numpy(data_test).float()
test_loader = DataLoader(TensorDataset(data), batch_size=batch_size, shuffle=False)

### Classify objects

In [ ]:
def test(test_loader):
    '''
    Forward pass through the CNN for one epoch
    '''
    model.eval()
    predictions = np.array([], dtype=np.int64).reshape(0) # placeholder for all predictions
    scores = np.array([], dtype=np.float32).reshape(0,4) # placeholder for all scores
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            if cuda:
                data = Variable(data[0].cuda())
            else:
                data = Variable(data[0])
            output = model(data) # forward pass through the CNN
            pred = output.data.max(1)[1] # get the index of the max log-probability
            predictions = np.concatenate((predictions, pred.cpu().numpy()))
            scores = np.concatenate((scores, output.data.cpu().numpy()),axis=0)
    return predictions, scores


def load_weights(model):
    '''
    Load trained parameters to model
    '''
    model_dict = model.state_dict()
    if cuda:
        pretrained_dict = torch.load(checkpoint)
    else:
        pretrained_dict = torch.load(checkpoint, map_location=torch.device('cpu'))
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.size() == model_dict[k].size() }
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

## Create text file with predictions

In [ ]:
start_time = time.time()

cuda = cuda and torch.cuda.is_available()
os.environ["CUDA_VISIBLE_DEVICES"] = gpu

model = Net() # create model with StarcNet architecture
load_weights(model) # load trained model parameters

if cuda:
    model.cuda()

predictions, scores = test(test_loader) # classify all 32x32x5 arrays

np.save(os.path.join('output','scores'), scores) # save scores to 'output/scores.npy' file
print('End of classification | time: %.2fs'%(time.time() - start_time))
print('-'*40)
print('Objects classified as Class 1: %d'%(len(np.where(predictions == 0)[0])))
print('Objects classified as Class 2: %d'%(len(np.where(predictions == 1)[0])))
print('Objects classified as Class 3: %d'%(len(np.where(predictions == 2)[0])))
print('Objects classified as Class 4: %d'%(len(np.where(predictions == 3)[0])))


## OT:

In [ ]:
import matplotlib.pyplot as plt

def L0_1D(x, thresh):

    x_abs = np.abs(x)
    x_large = np.where(x_abs>thresh)[0]
    
    y = 0
    for i in range(x_large.shape[0]):
        if i == 1:
            if x_large[i]==1:
                y = y + 1
        else:
            if x_large[i]==1 and x_large[i-1]==0:
                y = y + 1
    return y

def L0_2D(x):
    x_abs = np.abs(x)
    thresh = np.amax(x_abs)*0.5
    y = 0
    for i in range(x_abs.shape[0]):
        for j in range(x_abs.shape[1]):
        
            if x_abs[i,j]>thresh:
                if i == 0 and j == 0:
                    y = y + 1
                elif i == 0:
                    if x_abs[i,j-1]<=thresh:
                        y = y + 1
                elif j == 0:
                    if x_abs[i-1,j]<=thresh and x_abs[i-1,j+1]<=thresh:
                        y = y + 1
                elif j == x_abs.shape[1]-1:
                    if x_abs[i-1,j]<=thresh and x_abs[i,j-1]<=thresh and x_abs[i-1,j-1]<=thresh:
                        y = y + 1                    
                else:
                    if x_abs[i-1,j]<=thresh and x_abs[i,j-1]<=thresh \
                            and x_abs[i-1,j-1]<=thresh and x_abs[i-1,j+1]<=thresh:
                        y = y + 1
    return y

def entropy1D(X):
    e = 0
    for i in range(X.shape[0]):
        if X[i]>0:
            e = e - (X[i] * np.log(X[i]))
    return e
    
def deterministic_dist(n,mesh_v):

    targets = np.zeros((n, (mesh_v.shape[0])**n))
    
    if n==1:
        c = 0
        for i1 in mesh_v:
            targets[:,c]=i1
            targets[:,c] = targets[:,c]/norm(targets[:,c],1)
            c = c + 1
    elif n==2:
        c = 0
        for i1 in mesh_v:
            for i2 in mesh_v:
                targets[:,c]=[i1, i2]
                targets[:,c] = targets[:,c]/norm(targets[:,c],1)
                c = c + 1
    elif n==3:
        c = 0
        for i1 in mesh_v:
            for i2 in mesh_v:
                for i3 in mesh_v:
                    targets[:,c]=[i1, i2, i3]
                    targets[:,c] = targets[:,c]/norm(targets[:,c],1)
                    c = c + 1
    elif n==4:
        c = 0
        for i1 in mesh_v:
            for i2 in mesh_v:
                for i3 in mesh_v:
                    for i4 in mesh_v:
                        targets[:,c]=[i1, i2, i3, i4]
                        targets[:,c] = targets[:,c]/norm(targets[:,c],1)
                        c = c + 1
    elif n==5:
        c = 0
        for i1 in mesh_v:
            for i2 in mesh_v:
                for i3 in mesh_v:
                    for i4 in mesh_v:
                        for i5 in mesh_v:
                            targets[:,c]=[i1, i2, i3, i4, i5]
                            targets[:,c] = targets[:,c]/norm(targets[:,c],1)
                            c = c + 1
    else:
        for i in range(targets.shape[1]):
            ind = np.array(np.unravel_index(i, (mesh_v.shape[0] for j in n)))
            for k in range(targets.shape[0]):
                targets[k,i] = mesh_v[ind[k]]
            targets[:,i] = targets[:,i]/norm(targets[:,i],1)
            
    return targets

def get_rand_peak(image_width, image_height):
    (X, Y) = np.meshgrid(np.linspace(-1,1,image_width), np.linspace(-1,1,image_height))
    sumXY = np.abs(X)+np.abs(Y)
    target_2d = np.maximum(1.- np.random.rand() * sumXY, 0.)
    x_shift = np.random.randint(0,image_width)
    y_shift = np.random.randint(0,image_height)
    target_2d = np.roll(target_2d, x_shift, axis=0)
    target_2d = np.roll(target_2d, y_shift, axis=1)
    if x_shift<image_width/2.:
        target_2d[0:x_shift, :] = 0
    else:
        target_2d[x_shift:image_width, :] = 0
    if y_shift<image_height/2.:
        target_2d[:, 0:y_shift] = 0
    else:
        target_2d[:, y_shift:image_height] = 0
    return target_2d

def predict_OT(image, lambda_v, channel):

    image_width = image.shape[1]
    image_height = image.shape[2]
    source = image[channel,:,:].flatten()
    source = source/norm(source,1)

    n = source.shape[0]
    epsilon = .01

    C = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            (ix, iy) = np.unravel_index(i,(image_width, image_height))
            (jx, jy) = np.unravel_index(j,(image_width, image_height))
            C[i,j] = norm([ix-jx, iy-jy])

    K = np.exp(-C/epsilon)

#     mesh_size = 3
#     mesh_v = np.linspace(0,1,num=mesh_size)
#     targets_size = (mesh_v.shape[0])**n
    targets_size = 20
    
    dist_W = np.zeros((targets_size,lambda_v.shape[0]))
    dist_l1 = np.zeros((targets_size,lambda_v.shape[0]))
    dist_l2 = np.zeros((targets_size,lambda_v.shape[0]))
    target_v = np.zeros((targets_size,lambda_v.shape[0],n))
    target_W_entropy = np.zeros((lambda_v.shape[0]))
    target_W_points = np.zeros((lambda_v.shape[0]))

    for lambda_ind in range(lambda_v.shape[0]):
#         targets = deterministic_dist(n,mesh_v)
#         for rand_ind in range(targets_size):
#             target = targets[:,rand_ind]
        for rand_ind in range(targets_size):
            print(str(rand_ind)+',', end = '')
            
            if np.random.rand() > .5:
                target_2d = get_rand_peak(image_width, image_height)
            else:
                target_2d = get_rand_peak(image_width, image_height) \
                            + get_rand_peak(image_width, image_height)
                
            target = target_2d.flatten()
            target = target/norm(target,1)

            u = np.ones(n)
            v = np.ones(n)
            T = np.diag(u) @ K @ np.diag(v)

            for opt_ind in range(100):
                u = source/(K @ v)
                v = target/(K.T @ u)
                T = np.diag(u) @ K @ np.diag(v)

            dist_W[rand_ind,lambda_ind] = norm(C * T, 'fro') \
                + lambda_v[lambda_ind]*entropy1D(target)
            dist_l1[rand_ind,lambda_ind] = norm(source-target,1) \
                + lambda_v[lambda_ind]*entropy1D(target)
            dist_l2[rand_ind,lambda_ind] = norm(source-target,2) \
                + lambda_v[lambda_ind]*entropy1D(target)
            target_v[rand_ind,lambda_ind,:] = target.flatten()

        print('')            
        I = np.argmin(dist_W[:,lambda_ind])
        optimal_target = target_v[I,lambda_ind,:]
        target_W_entropy[lambda_ind] = entropy1D(optimal_target)
        target_W_points[lambda_ind] = L0_2D(np.reshape(optimal_target, (image_width, image_height)))
    print(target_W_entropy)
    print(target_W_points)
    return target_W_points[0]
    
def test_OT(test_loader, predictions_shape, scores_shape, lambda_v, channel, predictions_nnet, 
            data_subsample, data_np):
    prediction_class = 3+np.zeros(predictions_shape, dtype=np.int64)
    
    for i in data_subsample:
        p = predict_OT(data_np[i,:,:,:], lambda_v, channel)
        print('data point %d, true class label %d, prediction %f'%(i, 1+predictions_nnet[i], p))

        if p==1:
            prediction_class[i] = 0
        else:
            prediction_class[i] = 1

    return prediction_class, None

In [ ]:
print('OT:')
data_subsample = np.random.choice(predictions.shape[0],size=10,replace=False)
data_np = data.numpy()
lambda_v = 10.**np.arange(-5,1,1)
for lambda_ind in range(lambda_v.shape[0]):
    print('lambda: %2.3f'%lambda_v[lambda_ind])
    predictions_OT = {}
    for i in data_subsample:
        if predictions[i]==1: # or predictions[i]==0:
            for channel in range(0,5):
                plt.matshow(data_np[i,channel,:,:])
                plt.title('data point: %d, channel: %d, class: %d'%(i, channel, predictions[i]+1))
                print('channel: %d'%channel)
                prediction_OT, score_OT = test_OT(test_loader, predictions.shape, scores.shape, 
                                                  np.array([lambda_v[lambda_ind]]), channel, 
                                                  predictions, [i], data_np)
                print('')
                predictions_OT[i] = prediction_OT[i]
                
    prediction_OT = 0*prediction_OT+3
    for k,v in predictions_OT.items():
        prediction_OT[k] = v
    print('Objects classified as Class 1: %d'%(len(np.where(prediction_OT == 0)[0])))
    print('Objects classified as Class 2: %d'%(len(np.where(prediction_OT == 1)[0])))
    print('Objects classified as Class 3: %d'%(len(np.where(prediction_OT == 2)[0])))
    print('Objects classified as Class 4: %d'%(len(np.where(prediction_OT == 3)[0])))
    predictions_agree = np.where(predictions == prediction_OT)
    print('Class 1 accuracy: %f'%(len(np.where(predictions[predictions_agree] == 0)[0])/predictions.shape[0]))
    print('Class 2 accuracy: %f'%(len(np.where(predictions[predictions_agree] == 1)[0])/predictions.shape[0]))
    print('Class 3 accuracy: %f'%(len(np.where(predictions[predictions_agree] == 2)[0])/predictions.shape[0]))
    print('Class 4 accuracy: %f'%(len(np.where(predictions[predictions_agree] == 3)[0])/predictions.shape[0]))
                

In [ ]:
with open('data/test_raw_32x32.dat', 'rb') as infile:
    dset = pickle.load(infile)
data, ids, galaxies, coords = dset['data'], dset['ids'], dset['galaxies'], dset['coordinates']

scores = np.load('output/scores.npy')
preds = np.argmax(scores,axis=1)

with open('output/predictions.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Galaxy', 'Id', 'X', 'Y','Prediction'])
    for i in range(len(ids)):
        filewriter.writerow([galaxies[i], ids[i], coords[i][0], coords[i][1], preds[i]+1])

print("Text file with predictions: 'output/predictions.csv'")

## Create galaxy image with predictions

In [ ]:
os.system('python src/run_visualization.py')

print("Image with predictions saved to 'output/visualizations/' folder")

## Visualize top 10 classified objects per class

In [ ]:
import matplotlib.pyplot as plt
from scipy.special import softmax

def legus2rgb(im):
    datac = np.zeros((im.shape[1],im.shape[2],3), dtype=np.float64)
    datac[:,:,2] = (21.63*im[0,:,:] + 8.63*im[1,:,:]) / 2.
    datac[:,:,1] = (4.52*im[2,:,:])
    datac[:,:,0] = (1.23*im[3,:,:] + im[4,:,:]) / 2.
    return np.clip(datac, 0,1)

In [ ]:
sz = 16 # patch size (i.e. sz x sz pixels)
num_objs = 10 # top-N objects to show

for class_label in range(4):
    class_ids = np.where(predictions == class_label)
    if len(class_ids[0]) == 0: 
        print('[Class %d] No objects predicted'%(class_label+1))
        continue
    scores_ids = softmax(scores, axis=1)[class_ids][:,class_label]
    sorted_ids = np.argsort(scores_ids)[::-1]
    
    print('[Class %d] Objects: %d'%(class_label+1,len(sorted_ids)))
    fig = plt.figure(figsize=(14, 4), dpi= 80, facecolor='w', edgecolor='k')
    for i in range(num_objs):
        if i >= len(class_ids[0]): continue
        plt.subplot(1,num_objs,i+1)
        plt.imshow(legus2rgb(data_all[class_ids][sorted_ids][i,:,:,:]))
        plt.title('%d(%.1f%%)'%(ids[class_ids[0][sorted_ids[i]]],scores_ids[sorted_ids[i]]*100))
        plt.axis('off')
    plt.show()
    
# Each image has as title the id and score [id(score)]